In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

from expected_tackling.data.mott_features import compute_mott_features_data

In [3]:
features_data = pd.concat([pd.read_csv(f"../data/features_week_{i}.csv") for i in range(1,10)]).reset_index(drop=True)
tackling_probability = pd.read_csv("../data/tackling_probability.csv")
tackles = pd.read_csv("../data/tackles.csv")

In [4]:
mott_features_data = compute_mott_features_data(features_data, tackling_probability, tackles)

In [5]:
mott_features_data

tackling_probability  distance_to_ball_carrier  \
gameId     playId nflId                                                     
2022090800 56     38577.0              0.363425                  2.295321   
                  41239.0              0.008615                 14.187872   
                  42816.0              0.037124                 29.415605   
                  43294.0              0.701456                  0.670298   
                  43298.0              0.003791                 12.298996   
...                                         ...                       ...   
2022110700 3787   52627.0              0.228023                 18.914695   
                  53460.0              0.374259                  3.238673   
                  53533.0              0.021858                 19.715603   
                  54541.0              0.879435                  0.773886   
                  54606.0              0.070265                 13.049031   

                           frame_progression  nb_frames_from_end  \
gameId     playId nflId                                            
2022090800 56     38577.0           0.812500                 3.0   
                  41239.0           0.500000                 8.0   
                  42816.0           0.250000                12.0   
                  43294.0           1.000000                 0.0   
                  43298.0           0.000000                16.0   
...                                      ...                 ...   
2022110700 3787   52627.0           0.558824                15.0   
                  53460.0           0.500000                17.0   
                  53533.0           1.000000                 0.0   
                  54541.0           0.794118                 7.0   
                  54606.0           0.911765                 3.0   

                           last_distance_to_ball_carrier  \
gameId     playId nflId                                    
2022090800 56     38577.0                       1.695789   
                  41239.0                      13.245384   
                  42816.0                      25.538068   
                  43294.0                       0.670298   
                  43298.0                      11.767770   
...                                                  ...   
2022110700 3787   52627.0                       8.332647   
                  53460.0                       6.202064   
                  53533.0                      19.715603   
                  54541.0                       1.462771   
                  54606.0                      12.336296   

                           last_tackling_probability  \
gameId     playId nflId                                
2022090800 56     38577.0                   0.228518   
                  41239.0                   0.000750   
                  42816.0                   0.001556   
                  43294.0                   0.701456   
                  43298.0                   0.001199   
...                                              ...   
2022110700 3787   52627.0                   0.028729   
                  53460.0                   0.047986   
                  53533.0                   0.021858   
                  54541.0                   0.766604   
                  54606.0                   0.060141   

                           ball_carrier_distance_won_to_last_frame  \
gameId     playId nflId                                              
2022090800 56     38577.0                                     0.00   
                  41239.0                                     0.00   
                  42816.0                                     0.52   
                  43294.0                                     0.00   
                  43298.0                                     1.15   
...                                                            ...   
2022110700 3787   52627.0                                     5.57   


In [6]:
sample_mott_features_data = pd.concat([
    mott_features_data[mott_features_data["pff_missedTackle"]==1],
    mott_features_data[mott_features_data["pff_missedTackle"]==0].sample(
        len(mott_features_data[mott_features_data["pff_missedTackle"]==1])*10
    )
])

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, balanced_accuracy_score
from catboost import CatBoostClassifier

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    sample_mott_features_data.drop(columns=["pff_missedTackle"]), 
    sample_mott_features_data["pff_missedTackle"], 
    test_size=0.33, 
    random_state=42
)

In [31]:
model = CatBoostClassifier(max_depth=3, scale_pos_weight=5, verbose=0)
model.fit(X_train, y_train)

In [32]:
y_pred = model.predict(X_test)

In [33]:
y_test.sum()/len(y_test)

0.08632490571308842

In [34]:
y_pred.sum()/len(y_pred)

0.10588070959631234

In [35]:
balanced_accuracy_score(y_train, model.predict(X_train))

0.9477778308290695

In [36]:
balanced_accuracy_score(y_test, y_pred)

0.9308551140453867

In [37]:
confusion_matrix(y_train, model.predict(X_train))

array([[12844,   335],
       [  107,  1247]], dtype=int64)

In [38]:
confusion_matrix(y_test, y_pred)

array([[6335,  206],
       [  66,  552]], dtype=int64)

In [39]:
import pickle
pickle.dump(model, open("../models/model_mott.pkl", 'wb'))

In [40]:
mott_features_data.to_csv("../data/features_mott.csv")